In [4]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import_ipynb

In [5]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:

from tensorflow.keras.models import load_model
import argparse
import numpy as np
import sys
from Bio.PDB import *

# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path
%cd "/content/drive/MyDrive/Ex4files/" 
import import_ipynb
import utils


/content/drive/MyDrive/Ex4files


In [7]:
###############################################################################
#                                                                             #
#                        Change these parameters                              #
#                                                                             #
###############################################################################

DIST_STD = 1.3 #1
OMEGA_STD = 0.85 #4
THETA_STD = 0.25 #3
PHI_STD = 0.18 #2


utils files

In [8]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Ex4files/"

from Bio.PDB import *
import numpy as np
import re
import os
import pickle
from tqdm import tqdm

W_RPattern = re.compile(r'W\wR')
Y_CPattern = re.compile(r'(Y\wC)|(YY\w)')
WG_GPattern = re.compile(r'(WG\wG)|(W\wQG)|(\wGQG)|(\w\wQG)|(WG\w\w)|(W\w\wG)|(W\wQ\w)')
CDR_MAX_LENGTH = 32
AA_DICT = {"A": 0, "C": 1, "D": 2, "E": 3, "F": 4, "G": 5, "H": 6, "I": 7, "K": 8, "L": 9, "M": 10, "N": 11,
           "P": 12, "Q": 13, "R": 14, "S": 15, "T": 16, "W": 17, "Y": 18, "V": 19, "-": 20, "X": 20}


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Ex4files


In [9]:
def find_cdr3(sequence):
    """
    find the start and end indexes of the CDR3 in the given nanobody sequence
    :param sequence: String
    :return: [start, end]  (ints)
    """
    left_area = sequence[90:105]
    la_i = -1
    left_cdr = -1
    Y_C = Y_CPattern.search(left_area)
    if Y_C != None:
        # if we found 'YXR', find its index
        la_i = Y_C.start(0)+2
    else:
        la_i = left_area.find('C')  # didn't find 'YXC', look for 'C'

    if la_i >= 0:
        left_cdr = la_i + 90 + 3
    n = len(sequence) - 1
    n1 = n - 14
    subtract_amount = 1
    right_area = sequence[n1:n - 4]
    ra_i = -1
    right_cdr = -1
    WG_G = WG_GPattern.search(right_area)
    if WG_G != None:
        ra_i = WG_G.start(0)

    if ra_i >= 0:
        right_cdr = ra_i + n1 - subtract_amount + 1  # CDR3 ends at 'W' - 1 (or 'Q' - 3) (add n-14 to put it back in the full sequence)
    # check
    if left_cdr == -1 and right_cdr == -1:
        left_cdr = n - 21
        right_cdr = n - 10
    elif left_cdr == -1:
        left_cdr = right_cdr - 11
    elif right_cdr == -1:
        if left_cdr + 11 <= n:
            right_cdr = left_cdr + 11
        else:
            right_cdr = n
    if left_cdr > right_cdr:
        left_cdr = n - 1
        right_cdr = n
    return [left_cdr,right_cdr]


In [10]:
def get_seq_aa(chain):
    """
    returns the sequence (String) and a list of all the aa residue objects of the given nanobody chain.
    :param chain: BioPython chain object
    :return: sequence, [aa objects]
    """
    aa_residues = []
    seq = ""

    for residue in chain.get_residues():
        aa = residue.get_resname()
        if not is_aa(aa) or not residue.has_id('CA'):
            continue
        elif aa == "UNK":
            seq += "X"
            aa_residues.append(residue)
        else:
            seq += Polypeptide.three_to_one(residue.get_resname())
            aa_residues.append(residue)

    return seq, aa_residues

In [11]:
def generate_input(pdb_file):
    """
    receives a pdb file and returns its CDR3 sequence in a one-hot encoding matrix (each row is an aa in the sequence, and
    each column represents a different aa out of the 20 aa + 1 special column).
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H')
    :return: numpy array of size (CDR_MAX_LENGTH * 21)
    """
    # load model
    model = PDBParser().get_structure(pdb_file, pdb_file)[0]["H"]

    # get seq and aa residues
    seq, aa_residues = get_seq_aa(model)

    # find cdr3 start and end
    [cdr_start, cdr_end] = find_cdr3(seq)
    cdr_len = (cdr_end + 1 - cdr_start)

    cdr_up_pad = (CDR_MAX_LENGTH - cdr_len) // 2
    cdr_down_pad = CDR_MAX_LENGTH - cdr_up_pad - cdr_len

    # pad the sequence with '-'
    seq_cdr = cdr_up_pad * "-" + seq[cdr_start:cdr_end + 1] + cdr_down_pad * "-"

    # turn in to one-hot encoding matrix
    cdr_matrix = np.zeros((CDR_MAX_LENGTH, 21))
    for i in range(CDR_MAX_LENGTH):
        cdr_matrix[i][AA_DICT[seq_cdr[i]]] = 1

    return cdr_matrix
    

In [12]:
def write_const_dist(const_file, constraints, cdr3_s, seq):
    """
    writes the distance constraints into const_file in Rosetta format, uses harmonic function
    :param const_file: file to write the constraints into
    :param constraints: constraints matrix with no padding
    :param cdr3_s: start index of cdr3
    :param seq: nanobody sequence
    :return: None
    """
    length = len(constraints)
    for i in range(length):
        for j in range(i+1, length):  # symmetry
            atom_i = i + cdr3_s
            atom_j = j + cdr3_s

            atom_i_type = 'CA' if seq[atom_i] == 'G' else 'CB'  # GLY
            atom_j_type = 'CA' if seq[atom_j] == 'G' else 'CB'  # GLY

            atom_i += 1  # pdb numbering starts from 1
            atom_j += 1  # pdb numbering starts from 1

            const_file.write("AtomPair {} {} {} {} HARMONIC {:.5f} {}\n".format(atom_i_type, atom_i, atom_j_type, atom_j, constraints[i,j], DIST_STD))

In [13]:
def write_const_omega(const_file, constraints, cdr3_s, seq):
    """
   writes the omega constraints into const_file in Rosetta format, uses circular harmonic function
   :param const_file: file to write the constraints into
   :param constraints: constraints matrix with no padding
   :param cdr3_s: start index of cdr3
   :param seq: nanobody sequence
   :return: None
   """
    length = len(constraints)
    for i in range(length):
        for j in range(i + 1, length):  # symmetry
            atom_i = i + cdr3_s
            atom_j = j + cdr3_s
            if seq[atom_i] == 'G' or seq[atom_j] == 'G':  # GLY
                continue
            atom_i += 1  # pdb numbering starts from 1
            atom_j += 1  # pdb numbering starts from 1

            const_file.write("Dihedral CA {} CB {} CB {} CA {} CIRCULARHARMONIC {:.5f} {}\n".format(atom_i, atom_i, atom_j, atom_j, constraints[i, j], OMEGA_STD))


In [14]:
def write_const_theta(const_file, constraints, cdr3_s, seq):
    """
   writes the theta constraints into const_file in Rosetta format, uses circular harmonic function
   :param const_file: file to write the constraints into
   :param constraints: constraints matrix with no padding
   :param cdr3_s: start index of cdr3
   :param seq: nanobody sequence
   :return: None
   """
    length = len(constraints)
    for i in range(length):
        for j in range(length):
            if i == j:  # same atom...
                continue
            atom_i = i + cdr3_s
            atom_j = j + cdr3_s
            if seq[atom_i] == 'G' or seq[atom_j] == 'G':  # GLY
                continue
            atom_i += 1  # pdb numbering starts from 1
            atom_j += 1  # pdb numbering starts from 1
            const_file.write("Dihedral N {} CA {} CB {} CB {} CIRCULARHARMONIC {:.5f} {}\n".format(atom_i, atom_i, atom_i, atom_j, constraints[i, j], THETA_STD))


In [15]:
def write_const_phi(const_file, constraints, cdr3_s, seq):
    """
   writes the phi constraints into const_file in Rosetta format, uses circular harmonic function
   :param const_file: file to write the constraints into
   :param constraints: constraints matrix with no padding
   :param cdr3_s: start index of cdr3
   :param seq: nanobody sequence
   :return: None
   """
    length = len(constraints)
    for i in range(length):
        for j in range(length):
            if i == j:  # same atom...
                continue
            atom_i = i + cdr3_s
            atom_j = j + cdr3_s
            if seq[atom_i] == 'G' or seq[atom_j] == 'G':  # GLY
                continue

            atom_i += 1  # pdb numbering starts from 1
            atom_j += 1  # pdb numbering starts from 1

            const_file.write("Angle CA {} CB {} CB {} CIRCULARHARMONIC {:.5f} {}\n".format(atom_i, atom_i, atom_j, constraints[i, j], PHI_STD))

In [16]:
def remove_padding(matrix, cdr_length):
    """
    removes the zero padding added to a numpy array using add_padding(), cdr_length is the size of the matrix before
    calling add_padding().
    :param matrix: numpy array of size (CDR_MAX_LENGTH * CDR_MAX_LENGTH * m)
    :param cdr_length: the original size of the matrix (the cdr3 length), int
    :return: a numpy array of size (cdr_length * cdr_length * m)
    """
    pad = (CDR_MAX_LENGTH - (cdr_length)) // 2
    return matrix[pad:pad + cdr_length, pad:pad + cdr_length]

In [17]:
def write_const_file(pdb_sequence, constraints_list, output_file):
    """
    writes the constraints file in Rosetta format.
    :param pdb_sequence: nanobody sequence (String)
    :param constraints_list: list of constraints obtained by using predict
    :return: None
    """
    cdr_start, cdr_end = find_cdr3(pdb_sequence)
    cdr_len = (cdr_end + 1) - cdr_start

    # remove padding
    distance_const = remove_padding(constraints_list[0][0], cdr_len)
    omega_const = remove_padding(constraints_list[1][0], cdr_len)
    theta_const = remove_padding(constraints_list[2][0], cdr_len)
    phi_const = remove_padding(constraints_list[3][0], cdr_len)

    # retrieve the real distances and angles
    distance_const = distance_const[:, :, 0] * 10  # we divided by factor 10 in the processing of the data
    omega_const = np.arctan2(omega_const[:, :, 0], omega_const[:, :, 1])  # angle = arctan(sin, cos)
    theta_const = np.arctan2(theta_const[:, :, 0], theta_const[:, :, 1])  # angle = arctan(sin, cos)
    phi_const = np.arctan2(phi_const[:, :, 0], phi_const[:, :, 1])  # angle = arctan(sin, cos)

    with open(output_file, "w") as const_file:
        write_const_dist(const_file, distance_const, cdr_start, pdb_sequence)
        write_const_omega(const_file, omega_const, cdr_start, pdb_sequence)
        write_const_theta(const_file, theta_const, cdr_start, pdb_sequence)
        write_const_phi(const_file, phi_const, cdr_start, pdb_sequence)


In [18]:
def main(pdb_file, network, output_file):
    """
    this program recieves a path to a nanobody file and a trained network and creates a constraints file in the path <output_file> 
    """

    # load the trained model
    network_model = load_model(network)

    # get the sequence of the nanobody (so we can skip GLY when needed)
    pdb_chain = PDBParser().get_structure(pdb_file, pdb_file)[0]["H"]
    sequence, aa_chains = get_seq_aa(pdb_chain)

    # generate the constraints using the network you built
    pdb_constraints = network_model.predict(np.array([generate_input(pdb_file)]))

    # write the constraints into a file, replace the empty list with your list.
    write_const_file(sequence, pdb_constraints, output_file)

In [19]:
pdb = input("nanobody pdb file:")
# /content/drive/MyDrive/Ex4files/ref.pdb
network = input("trained model:")
# /content/drive/MyDrive/Ex4files/Ex4Output/our_model
output_file = input("path of output file:")
# /content/drive/MyDrive/Ex4files/Ex4Output/constraints/constraints_file
main(pdb, network, output_file)



nanobody pdb file:/content/drive/MyDrive/Ex4files/ref.pdb
trained model:/content/drive/MyDrive/Ex4files/Ex4Output/our_model
path of output file:/content/drive/MyDrive/Ex4files/Ex4Output/constraints/constraints_file
